In [1]:
SRC = 'https://www.kyoukaikenpo.or.jp/~/media/Files/shared/hokenryouritu/r2/ippan/r2030113tokyo.pdf'
DST = '2020-04.csv'

In [2]:
import datetime
import hashlib
import io
import pathlib
import urllib.request

try:
    import tabula
except ModuleNotFoundError:
    !pip install tabula-py
    import tabula

datetime.datetime.now().astimezone(datetime.timezone.utc).isoformat()

'2022-05-07T15:36:46.662555+00:00'

In [3]:
data = io.BytesIO()

with urllib.request.urlopen(SRC) as f:
    data.write(f.read())

hashlib.sha256(data.getvalue()).hexdigest()

'fe748404991bc1482763abbe9b7af3ef3115225c5a23289428a2c616a7aaca5c'

In [4]:
data.seek(0)

df = tabula.read_pdf(data, lattice=True, pages=1)[0]

df = df.iloc[:, [1, 2, 3, 4, 5, 6, 7]]

df.columns = """
標準報酬:等級
標準報酬:月額
報酬月額
介護保険の被保険者でない場合:全額
介護保険の被保険者でない場合:折半額
介護保険の被保険者の場合:全額
介護保険の被保険者の場合:折半額
""".strip().split()

df['標準報酬:等級'] = df['標準報酬:等級'].str.replace(r'\([0-9]+\)', '', regex=True)
df = df[~df['標準報酬:等級'].isnull()]
df = df[df['標準報酬:等級'].str.match(r'^[0-9]+$')]
df['標準報酬:等級'] = df['標準報酬:等級'].astype(int)
df = df.set_index('標準報酬:等級').sort_index()

assert df.shape[0] == 50, f'等級数 ({df.shape[0]}) に過不足があります。'

s = df['報酬月額'].str.replace(r'[^0-9~]', '', regex=True)
df_ = s.str.split('~', n=1, expand=True)
df_ = df_.rename(columns={0: '報酬月額:以上', 1: '報酬月額:未満'})
df = df.merge(df_, left_index=True, right_index=True)

del df['報酬月額']

for x in df.columns:
    df[x] = df[x].str.replace(',', '')

df = df.replace('', float('nan'))
df['報酬月額:以上'] = df['報酬月額:以上'].fillna(float('-inf')).astype(float)
df['報酬月額:未満'] = df['報酬月額:未満'].fillna(float('+inf')).astype(float)
df['標準報酬:月額'] = df['標準報酬:月額'].astype(int)

for x in """
介護保険の被保険者でない場合:全額
介護保険の被保険者でない場合:折半額
介護保険の被保険者の場合:全額
介護保険の被保険者の場合:折半額
""".strip().split():
    df[x] = df[x].astype(float)

for x in """
介護保険の被保険者でない場合:
介護保険の被保険者の場合:
""".strip().split():
    df[x + '控除額'] = (df[x + '折半額'] + .49).astype(int)

df = df["""
標準報酬:月額
報酬月額:以上
報酬月額:未満
介護保険の被保険者でない場合:全額
介護保険の被保険者でない場合:折半額
介護保険の被保険者でない場合:控除額
介護保険の被保険者の場合:全額
介護保険の被保険者の場合:折半額
介護保険の被保険者の場合:控除額
""".strip().split()]

df.to_csv(DST)

hashlib.sha256(pathlib.Path(DST).read_bytes()).hexdigest()

Got stderr: 5月 08, 2022 12:36:47 午前 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored
5月 08, 2022 12:36:47 午前 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored



'f74a118a4775b65add43b9d38bd7a7f4d6473b15e2fc2257a52ab66478a056cc'